In [6]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# Masses (kg)
m_base = 1.964
m_arm = 0.248
m_prop = 0.024
m_camera = 0.050

# Base inertia (kg·m²) - note the base has a pose offset
I_base = np.diag([0.00414594, 0.00314594, 0.00534206])
base_pose = [0, 0, -0.0050916496945010194, 0, 0, 0]  # x,y,z,r,p,y

# Arm inertia (kg·m²) in arm frame (aligned with cylinder axis)
I_arm_local = np.array([
    [0.00085084, 0, 2.27096e-05],
    [0, 0.00087355, 0],
    [2.27096e-05, 0, 0.00011721]
])

# Propeller inertia (kg·m²) in propeller frame
I_prop_local = np.diag([4.25e-06, 0.00010145, 0.0001053])

# Camera inertia (kg·m²) in camera frame
I_camera_local = np.diag([0.00004, 0.00004, 0.00004])

def euler_to_quat(roll, pitch, yaw):
    """Convert Euler angles (roll, pitch, yaw) to quaternion [x,y,z,w]"""
    return R.from_euler('xyz', [roll, pitch, yaw]).as_quat()

# Define all bodies with their poses relative to base_dummy
bodies = {
    "base": {
        "mass": m_base,
        "inertia": I_base,
        "pose": base_pose + [1]  # x,y,z,roll,pitch,yaw,w (w=1 for no rotation)
    },
    # Arms and propellers (6 arms, 6 propellers)
    "arm_1": {
        "mass": m_arm,
        "inertia": I_arm_local,
        "pose": [0.1855, 0, 0, 0, 0, 0, 1]  # Along x-axis
    },
    "prop_1": {
        "mass": m_prop,
        "inertia": I_prop_local,
        "pose": [0.1855 + 0.0945, 0, 0.02, 0, 0, 0, 1]  # Offset from arm_1
    },
    "arm_2": {
        "mass": m_arm,
        "inertia": I_arm_local,
        "pose": [0.09275, 0.160643, 0, 0, 0, 1.047, 0.5]  # 60° rotation (approx)
    },
    "prop_2": {
        "mass": m_prop,
        "inertia": I_prop_local,
        "pose": [0.09275 + 0.0945 * np.cos(1.047), 
                 0.160643 + 0.0945 * np.sin(1.047), 
                 0.02, 
                 0, 0, 1.047, 0.5]
    },
    "arm_3": {
        "mass": m_arm,
        "inertia": I_arm_local,
        "pose": [-0.09275, 0.160643, 0, 0, 0, 2.094, 0.5]  # 120° rotation
    },
    "prop_3": {
        "mass": m_prop,
        "inertia": I_prop_local,
        "pose": [-0.09275 + 0.0945 * np.cos(2.094), 
                 0.160643 + 0.0945 * np.sin(2.094), 
                 0.02, 
                 0, 0, 2.094, 0.5]
    },
    "arm_4": {
        "mass": m_arm,
        "inertia": I_arm_local,
        "pose": [-0.1855, 0, 0, 0, 0, 3.14159, 1]  # 180° rotation
    },
    "prop_4": {
        "mass": m_prop,
        "inertia": I_prop_local,
        "pose": [-0.1855 - 0.0945, 0, 0.02, 0, 0, 3.14159, 1]
    },
    "arm_5": {
        "mass": m_arm,
        "inertia": I_arm_local,
        "pose": [-0.09275, -0.160643, 0, 0, 0, -2.094, 0.5]  # -120° rotation
    },
    "prop_5": {
        "mass": m_prop,
        "inertia": I_prop_local,
        "pose": [-0.09275 + 0.0945 * np.cos(-2.094), 
                 -0.160643 + 0.0945 * np.sin(-2.094), 
                 0.02, 
                 0, 0, -2.094, 0.5]
    },
    "arm_6": {
        "mass": m_arm,
        "inertia": I_arm_local,
        "pose": [0.09275, -0.160643, 0, 0, 0, -1.047, 0.5]  # -60° rotation
    },
    "prop_6": {
        "mass": m_prop,
        "inertia": I_prop_local,
        "pose": [0.09275 + 0.0945 * np.cos(-1.047), 
                 -0.160643 + 0.0945 * np.sin(-1.047), 
                 0.02, 
                 0, 0, -1.047, 0.5]
    },
    "camera": {
        "mass": m_camera,
        "inertia": I_camera_local,
        "pose": [0, 0, -0.025, 0, 0, 0, 1]
    }
}

# Compute CoM
com = np.zeros(3)
total_mass = 0
for name, body in bodies.items():
    mass = body["mass"]
    pos = np.array(body["pose"][:3])
    com += mass * pos
    total_mass += mass
com /= total_mass
print(f"Center of Mass: {com}")

# Compute total inertia at CoM
I_total = np.zeros((3, 3))
for name, body in bodies.items():
    mass = body["mass"]
    pos = np.array(body["pose"][:3]) - com  # Position relative to CoM
    
    # Handle quaternion (last 4 elements of pose)
    quat = body["pose"][3:]
    if len(quat) == 4:  # Already a quaternion
        rot = R.from_quat(quat).as_matrix()
    else:  # Assume Euler angles (roll, pitch, yaw)
        rot = R.from_euler('xyz', quat[:3]).as_matrix()
    
    # Transform inertia to base frame
    I_body_local = body["inertia"]
    I_body_global = rot @ I_body_local @ rot.T  # Rotate inertia
    
    # Parallel axis theorem
    r_skew = np.array([
        [0, -pos[2], pos[1]],
        [pos[2], 0, -pos[0]],
        [-pos[1], pos[0], 0]
    ])
    I_body_global += mass * (pos.dot(pos) * np.eye(3) - np.outer(pos, pos))

    I_total += I_body_global

print("\nTotal Mass (kg):", total_mass)
print("Center of Mass (m):", com)
print("\nInertia Matrix at CoM (kg·m²):")
print(I_total)

# Compute inertia at origin (0,0,0)
I_origin = np.zeros((3, 3))
for name, body in bodies.items():
    mass = body["mass"]
    pos = np.array(body["pose"][:3])  # Position relative to origin
    
    # Handle rotation
    quat = body["pose"][3:]
    if len(quat) == 4:
        rot = R.from_quat(quat).as_matrix()
    else:
        rot = R.from_euler('xyz', quat[:3]).as_matrix()
    
    I_body_local = body["inertia"]
    I_body_global = rot @ I_body_local @ rot.T
    I_body_global += mass * (pos.dot(pos) * np.eye(3) - np.outer(pos, pos))
    I_origin += I_body_global

print("\nInertia Matrix at Origin (kg·m²):")
print(I_origin)

# Eigenvalues analysis
print("\nEigenvalues of Inertia Matrix at CoM:")
eigenvalues, eigenvectors = np.linalg.eig(I_total)
print("Eigenvalues:", eigenvalues)
print("Eigenvectors (columns):")
print(eigenvectors)

Center of Mass: [ 6.38576144e-07 -2.14104653e-18 -2.29566648e-03]

Total Mass (kg): 3.646000000000001
Center of Mass (m): [ 6.38576144e-07 -2.14104653e-18 -2.29566648e-03]

Inertia Matrix at CoM (kg·m²):
[[ 4.09156347e-02  5.65603638e-05 -6.49426913e-05]
 [ 5.65603638e-05  4.01682194e-02  1.31273152e-05]
 [-6.49426913e-05  1.31273152e-05  6.92074505e-02]]

Inertia Matrix at Origin (kg·m²):
[[ 4.09348494e-02  5.65603638e-05 -6.49373464e-05]
 [ 5.65603638e-05  4.01874341e-02  1.31273152e-05]
 [-6.49373464e-05  1.31273152e-05  6.92074505e-02]]

Eigenvalues of Inertia Matrix at CoM:
Eigenvalues: [0.06920761 0.04091975 0.04016395]
Eigenvectors (columns):
[[ 2.29454495e-03  9.97174418e-01 -7.50860485e-02]
 [-4.47581745e-04  7.50872628e-02  9.97176866e-01]
 [-9.99997267e-01  2.25445999e-03 -6.18608165e-04]]
